In [2]:
import pandas as pd
import numpy as np
import math
import scipy

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.figure_factory as ff

from IPython.display import display, HTML

init_notebook_mode(connected=True)

reg_survey = pd.read_excel("Shendi_baseline_survey_for_CC_project_in_Sudan_results_20171224_utf8_villages_corregidas.xlsx", encoding = 'utf_8')

# <center>Baseline survey results for the Locality of Shendi </center>
### <center>*Climate Change and Livelihoods situation of 15 villages located in Kabushiya Unit, Shendi Locality, River Nile State*</center>

In [3]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

#bc_wages.head()
#reg_csv.info()
#bc_wages.describe()

In [4]:
# Functions

# Get ocurrences of each value for a multiple_select field
def get_ocurrences(df, column_name):
    values_sums = df.groupby(column_name).size()
    values_names = reduce(lambda x, y: x + y,map(lambda x: x.split(), values_sums.index.values))
    result = pd.Series()
    for i in values_names:
        result[i] = values_sums.loc[[i in idx for idx in values_sums.index]].sum()
    return result

In [5]:
## File preprocessing
#Deleting 'demography:', 'house_assets:', 'incomes:', 'climate_change:' and those prefix which not provide useful info.

col_names = reg_survey.columns
col_names = col_names.map(lambda x: x[11:] if x.startswith("demography:") else x)
col_names = col_names.map(lambda x: x[13:] if x.startswith("house_assets:") else x)
col_names = col_names.map(lambda x: x[8:] if x.startswith("incomes:") else x)
col_names = col_names.map(lambda x: x[15:] if x.startswith("climate_change:") else x)
col_names = col_names.map(lambda x: x[11:] if x.startswith("intro_data:") else x)
col_names = col_names.map(lambda x: x[:11] + x[16:] if x.startswith("respondant_page:") else x)
col_names = col_names.map(lambda x: x[16:] if x.startswith("general_hh_data:") else x)
col_names = col_names.map(lambda x: x[6:] if x.startswith("males:") else x)
col_names = col_names.map(lambda x: x[8:] if x.startswith("females:") else x)
col_names = col_names.map(lambda x: x[11:] if x.startswith("importance:") else x)
col_names = col_names.map(lambda x: x[19:] if x.startswith("percentage_incomes:") else x)
col_names = col_names.map(lambda x: x[:10] + x[16:] if x.startswith("agri_gard:agri") else x)
col_names = col_names.map(lambda x: x[:10] + x[20:] if x.startswith("livestock:livestock_") else x)
col_names = col_names.map(lambda x: x[:10] + x[21:] if x.startswith("livestock:livestock") else x)
col_names = col_names.map(lambda x: x[:8] + x[17:] if x.startswith("fishing:fishing") else x)
col_names = col_names.map(lambda x: x[:7] + x[15:] if x.startswith("labour:labour") else x)
col_names = col_names.map(lambda x: x[:7] + x[14:] if x.startswith("labour:labour") else x)
col_names = col_names.map(lambda x: x[:12] + x[25:] if x.startswith("handicrafts:handicrafts") else x)
col_names = col_names.map(lambda x: x[:9] + x[19:] if x.startswith("forestry:forestry") else x)
col_names = col_names.map(lambda x: x[:6] + x[13:] if x.startswith("other:other") else x)
col_names = col_names.map(lambda x: x[7:] if x.startswith("coping:") else x)
col_names = col_names.map(lambda x: x[4:] if x.startswith(("cc1:", "cc2:", "cc3:", "ews:")) else x)
col_names = col_names.map(lambda x: x[10:] if x.startswith("renewable:") else x)
col_names = col_names.map(lambda x: x[12:] if x.startswith("agriculture_cc_effects:") else x)
col_names = col_names.map(lambda x: x[10:] if x.startswith("livestock_cc_effects:") else x)

print col_names.values

[u'deviceid' u'surveyor_name' u'number' u'village'
 u'geocoordinates:Latitude' u'geocoordinates:Longitude'
 u'geocoordinates:Altitude' u'geocoordinates:Accuracy' u'respondant_gender'
 u'respondant_age' u'respondant_marital_status' u'respondant_head_hh'
 u'education' u'handicapped_member' u'number_wifes' u'number_wifes_aux'
 u'male_infant' u'male_child' u'male_child_school' u'male_teenager'
 u'male_teenager_school' u'male_adult' u'male_elderly' u'female_infant'
 u'female_child' u'female_child_school' u'female_teenager'
 u'female_teenager_school' u'female_adult' u'female_elderly'
 u'family_males' u'family_females' u'family_total' u'family_confirm'
 u'house_material' u'belongings' u'kitchen_type' u'src_incomes' u'imp_agri'
 u'imp_garden' u'imp_livestock' u'imp_fishing' u'imp_labour'
 u'imp_forestry' u'imp_handcrafts' u'imp_other' u'perc_incomes_agri'
 u'perc_incomes_garden' u'perc_incomes_livestock' u'perc_incomes_fishing'
 u'perc_incomes_labour' u'perc_incomes_forestry' u'perc_incomes_ha

In [6]:
# Replace ':' with '_' for compatibility reasons.
col_names = col_names.map(lambda x: x[:14] + '_' + x[15:] if x.startswith("geocoordinates:") else x)
col_names = col_names.map(lambda x: x[:9] + '_' + x[10:] if x.startswith("agri_gard:") else x)
col_names = col_names.map(lambda x: x[:9] + '_' + x[10:] if x.startswith("livestock:") else x)
col_names = col_names.map(lambda x: x[:7] + '_' + x[8:] if x.startswith("fishing:") else x)
col_names = col_names.map(lambda x: x[:6] + '_' + x[7:] if x.startswith("labour:") else x)
col_names = col_names.map(lambda x: x[:11] + '_' + x[12:] if x.startswith("handicrafts:") else x)
col_names = col_names.map(lambda x: x[:8] + '_' + x[9:] if x.startswith("forestry:") else x)
col_names = col_names.map(lambda x: x[:5] + '_' + x[6:] if x.startswith("other:") else x)
col_names = col_names.map(lambda x: x[:10] + '_' + x[11:] if x.startswith("cc_effects:") else x)
col_names = col_names.map(lambda x: x[:4] + '_' + x[5:] if x.startswith("meta:") else x)
reg_survey.columns = col_names

In [7]:
# Processing NaN and void values

values = {
    'house_material': 'no_answer',
    'kitchen_type': 'no_answer',
    'use_ews': 'no_answer',
    'renewable_knowledge': 'no_answer',
    'solar_energy': 'no_answer',
    'livestock_selling': 'no_answer',
    'livestock_social': 'no_answer',
    'livestock_milk': 'no_answer',
    'livestock_meat': 'no_answer',
    'livestock_savings': 'no_answer',
    'labour_participants': 'no_answer',
    'agri_gard_water_harv_farmlands': 0,
    'agri_gard_land_quantity': 0,
    'cc_effects_sand_affected_farmlands': 'no_answer'
}
reg_survey.fillna(value=values, inplace=True)

In [8]:
# Creating villages dict
villages = reg_survey['village'].unique()
villages_table = pd.read_excel("villages_shendi.xlsx")
# Villages dictionary
villages_dict = villages_table.set_index('Arabic name').to_dict()['English transcription']
#print repr(villages_dict).decode('unicode_escape')


## General data

In [19]:
total_surveys = reg_survey['deviceid'].count()
print "Total:" + str(total_surveys)
print ""
surveys_village = reg_survey.groupby('village').size()
print surveys_village

Total:217

village
البواليد الستير     17
القهيد الخلوات      15
ام سكرة             12
ام عشيرة الشرقية    26
ام عشيرة الغربية    24
بئر السقيدي         12
بئر السلمانية       11
بئر الشين            5
بئر الفقيرة          6
بئر اليذيد           6
بئر بكوري           15
بئر ود قديم         14
شريشة               23
كوع السريح          16
وادي الدان          15
dtype: int64


Houses materials General and per village

In [9]:
#hey = reg_survey.groupby(['village', 'house_material']).size()

#house_materials_df = pd.DataFrame(index=house_materials_index, columns=villages, data=[house_materials_general])
#print house_materials_df

house_materials_df = pd.crosstab(reg_survey.village, reg_survey.house_material)
#display(house_materials_df)
#HTML(house_materials_df.to_html())

fig_house_materials = {
    "layout": {
        "title": {
            "font": {
                    "size": 20
            },
            "text": "House materials"
        },
        "annotations": [
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "All villages",
                "x": 0.17,
                "y": 1.1
            },
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "Village: " + house_materials_df.iloc[2].name + " (" + villages_dict[house_materials_df.iloc[2].name] + ")",
                "x": 0.93,
                "y": 1.1
            }
        ]
    },
    "data": [
        {

            "values": house_materials_df.sum().values,
            "labels": house_materials_df.columns.values,
            "type": "pie",
            "sort": False
        }
    ]
}

fig_house_materials2 = {
    "layout": {
        "title": {
            "font": {
                    "size": 20
            },
            "text": "House materials"
        }
    },
    "data": []
}

for i in range(len(villages_dict)-1):
    x = .2*(i%5)
    y = 0.35*(i/5)
    fig_house_materials2['data'].append(
        {

            "values": house_materials_df.iloc[i-1].values,
            "labels": house_materials_df.columns.values,
             "domain": {
                "x": [x, x+.18],
                 "y": [y, y+.3]
            },
            "type": "pie",
            "sort": False
        })

iplot(fig_house_materials)
iplot(fig_house_materials2)

###  Mean Household size

In [10]:
mean_hh_size = reg_survey.family_total.mean()
print mean_hh_size

4.01382488479


## Livelihoods

In [11]:
#reg_survey.loc[reg_survey['src_incomes'].str.contains("agriculture"), 'agriculture'] = 'yes'
#reg_survey.loc[reg_survey['src_incomes'].str.contains("livestock"), 'livestock'] = 'yes'
#reg_survey.loc[reg_survey['src_incomes'].str.contains("labour"), 'labour'] = 'yes'
#importance = reg_survey.groupby(['agriculture', 'livestock', 'labour'])['perc_incomes_agri'].agg('mean')

#agric = reg_survey[reg_survey['src_incomes'].str.contains("agriculture")]['deviceid'].count()

src_incomes_sums = reg_survey.groupby('src_incomes').size()
lh_values = get_ocurrences(reg_survey, 'src_incomes').sort_values(ascending = False)

print lh_values

display(src_incomes_sums)

labour         104
livestock      100
agriculture     57
other           39
handicrafts      3
forestry         1
dtype: int64


src_incomes
agriculture                           5
agriculture labour                    5
agriculture labour other              2
agriculture livestock                29
agriculture livestock forestry        1
agriculture livestock handicrafts     2
agriculture livestock labour          6
agriculture livestock other           3
agriculture other                     4
handicrafts                           1
labour                               75
labour other                          3
livestock                            41
livestock labour                     13
livestock other                       5
other                                22
dtype: int64

In [12]:
lh_perc = 100*lh_values/217
lh_perc = lh_perc.round(2)
livelihoods_strategies = {
    "layout": {
        "title": "Livelihoods Strategies",
        "xaxis": {
            "tickangle": -45
        },
        "width": 700,
        "annotations": []
    },
    "data": [
        {
            "x": map(lambda x: x.capitalize(), lh_perc.index.values),
            "y": lh_perc.values,
            "type": "bar",
            "name": "Livelihoods Strategies",
            "marker": {
                "color": 'rgb(258,225,225)',
                "line": {
                    "color": 'rgb(192,0,0)',
                    "width": 1.5
                }
            },
        }
    ]
}


for i in range(0, len(lh_values)):
    livelihoods_strategies["layout"]["annotations"].append(
        {
            "x": map(lambda x: x.capitalize(), lh_perc.index.values)[i],
            "y": lh_perc.values[i] + 1.5,
            "text": str(lh_perc.values[i]) + "%",
            "font": dict(
                family='Arial',
                size=14,
                color='rgba(100, 0, 0, 1)'
            ),
            "showarrow": False,

        }
    )

iplot(livelihoods_strategies)

### Livestock

In [13]:
livestock_set = reg_survey.loc[reg_survey['src_incomes'].str.contains("livestock")]

participants = livestock_set.groupby('livestock_participants').size()
participants_goats = livestock_set.loc[livestock_set['livestock_type'].str.contains("goat")].groupby('livestock_participants').size()
#print participants_goats

lvstk_types = ['camel', 'cattle', 'goats', 'sheep', 'chicken']

print get_ocurrences(livestock_set, 'livestock_type')
livestock_women = livestock_set.loc[livestock_set['livestock_participants'] == "women"]
type_women =  livestock_women.groupby('livestock_type').size()
#livestock_set = livestock_set.copy()
for i in lvstk_types:
    livestock_set[i + '_reduced'] = livestock_set['livestock_last_year_' + i] / (livestock_set['livestock_number_' + i] + livestock_set['livestock_number_' + i])
#print ("Types for women")
#print(type_women)
#print livestock_set.groupby('camel_reduced').size()
goats = livestock_set['livestock_number_goats']
q = goats.quantile([.3, .91, 1])
print q
sheeps = livestock_set.groupby('livestock_number_sheep').size()
camels = livestock_set.groupby('livestock_number_camel').size()
chickens = livestock_set.groupby('livestock_number_chicken').size()
prueba = pd.crosstab(livestock_set.livestock_number_goats, livestock_set.livestock_number_sheep).stack().reset_index(name='number')
#prueba.plot.scatter('livestock_number_goats', 'livestock_number_sheep', s=prueba.number * 10)
prueba = prueba[prueba.number != 0]
prueba = prueba[prueba.livestock_number_goats < 100]
prueba = prueba[prueba.livestock_number_sheep < 100]
#display(prueba)
#iplot(x=prueba.livestock_number_goats, y=prueba.livestock_number_sheep, size=prueba.number, text='Livestock',
#             xTitle='Number of goats', yTitle='Number of sheeps',
#             filename='cufflinks/simple-bubble-chart')
goats_vs_sheeps = {
    "layout": {
        "title": "Relation between the number of goats and sheeps"
    },
    "data": [
        {

            "x": prueba.livestock_number_goats,
            "y": prueba.livestock_number_sheep,
            "mode": "markers",
            "marker": {
                "size": prueba.number * 5               
            },
            "type": "scatter"
        }
    ]
}
iplot(goats_vs_sheeps)
goats1 = goats[goats < q[.3]].dropna()
goats2 = goats[goats.between(q[.3], q[.91])].dropna()
goats3 = goats[goats.between(q[.91], q[1])].dropna()
fig = ff.create_distplot([goats1.values, goats2.values, goats3.values], ['poor', 'medium', 'rich'])
iplot (fig)

camel      23
cattle      2
goat       89
sheep      67
chicken    13
other       5
dtype: int64
0.30     10.0
0.91     50.0
1.00    250.0
Name: livestock_number_goats, dtype: float64


In [14]:
sheeps1 = sheeps[sheeps < q[.3]].dropna()
sheeps2 = sheeps[sheeps.between(q[.3], q[.91])].dropna()
sheeps3 = sheeps[sheeps.between(q[.91], q[1])].dropna()
#fig = ff.create_distplot([sheeps1.values, sheeps2.values, sheeps3.values], ['poor', 'medium', 'rich'])
fig = ff.create_distplot([sheeps.values], ['sheeps'])
iplot (fig)

In [15]:
print livestock_set.groupby('cc_effects_grassland_availability').size()

cc_effects_grassland_availability
decreased        94
increased         1
no_changes        3
unpredictable     1
unsteady          1
dtype: int64


In [16]:
livestock_uses = ["selling", "social", "milk", "meat", "savings"]
livestock_uses_df = livestock_set[map(lambda x: "livestock_" + x, livestock_uses)]
livestock_uses_df.fillna('no')
print livestock_uses_df.apply(pd.Series.value_counts)

           livestock_selling  livestock_social  livestock_milk  \
no                         3                66             3.0   
no_answer                  1                 1             NaN   
yes                       96                33            97.0   

           livestock_meat  livestock_savings  
no                   30.0                  2  
no_answer             NaN                  1  
yes                  70.0                 97  


### Laborers

In [17]:
labour_set = reg_survey.loc[reg_survey['src_incomes'].str.contains("labour")]
print get_ocurrences(labour_set, 'labour_sector')
labour_participants =  labour_set.groupby('labour_participants').size()
labour_set.loc[labour_set['labour_participants'].str.contains('women')][['labour_participants', 'respondant_gender', 'respondant_age', 'respondant_head_hh', 'male_teenager', 'male_adult', 'male_elderly', 'labour_sector']]
labour_incomes = labour_set.groupby('labour_daily_salary').size()
mean_salary = labour_set['labour_daily_salary'].mean()
labour_incomes_others = labour_set.loc[labour_set['labour_sector'] == 'other'].groupby('labour_daily_salary').size()
sectors_100 = labour_set.loc[labour_set['labour_daily_salary'] == 100].groupby('labour_sector').size()
labor_term = labour_set.groupby('labour_work_term').size()
print labor_term

agriculture       12
government         2
livestock          5
other             78
transportation     6
dtype: int64
labour_work_term
permanent    42
seasonal     30
temporary    28
dtype: int64


### Agriculture

In [38]:
agri_set = reg_survey.loc[reg_survey['src_incomes'].str.contains("agriculture")]

print agri_set['deviceid'].count()

# Inconsistencies
agri_set.loc[agri_set.agri_gard_water_harv_farmlands > agri_set.agri_gard_land_quantity]['agri_gard_water_harv_farmlands'] = agri_set['agri_gard_land_quantity']
print agri_set.loc[agri_set.agri_gard_water_harv_farmlands > agri_set.agri_gard_land_quantity]['agri_gard_water_harv_farmlands']
print agri_set.loc[agri_set.agri_gard_water_harv_farmlands > agri_set.agri_gard_land_quantity]['agri_gard_land_quantity']
agri_set.loc[agri_set.agri_gard_water_harv_farmlands > agri_set.agri_gard_use_quantity]['agri_gard_water_harv_farmlands'] = agri_set['agri_gard_use_quantity']
print agri_set.loc[agri_set.agri_gard_water_harv_farmlands > agri_set.agri_gard_use_quantity]['agri_gard_water_harv_farmlands']
print agri_set.loc[agri_set.agri_gard_water_harv_farmlands > agri_set.agri_gard_use_quantity]['agri_gard_use_quantity']

agri_set['perc_rainfed_farmlands'] = agri_set['agri_gard_water_harv_farmlands'] / agri_set['agri_gard_land_quantity']

# Estimation to correct the inconsistencies: All feddan have water harvesting access
wh_feddans = agri_set['agri_gard_water_harv_farmlands'].sum() - 171

print wh_feddans / agri_set['deviceid'].count()
print wh_feddans / total_surveys

wh_feddans_poor_medium = agri_set_poor_medium['agri_gard_water_harv_farmlands'].sum()

print wh_feddans_poor_medium / agri_set_poor_medium['deviceid'].count()
print wh_feddans_poor_medium / total_surveys

agri_set_poor_medium = agri_set.loc[agri_set.agri_gard_use_quantity <= 10].copy()

print agri_set.groupby('perc_rainfed_farmlands').size()
print agri_set.groupby('agri_gard_farm_tech').size()
print agri_set_poor_medium.groupby('agri_gard_water_harv_farmlands').size()
all_farm_rainfed = agri_set.loc[agri_set['perc_rainfed_farmlands'] == 1].groupby('agri_gard_land_quantity').size()
#print agri_set.groupby('agri_gard_land_quantity').size()
print agri_set_poor_medium.groupby('agri_gard_use_quantity').size()
agri_set['agri_gard_water_harv_farmlands'].sum()
agri_set['agri_gard_land_quantity'].sum()
print agri_set.groupby('agri_gard_land_property').size()
print agri_set.groupby('cc_effects_sand_affected_farmlands').size()

agri_set.groupby('agri_gard_training').size()

57
51      10.0
55     100.0
162      1.0
194     60.0
Name: agri_gard_water_harv_farmlands, dtype: float64
51     0.0
55     0.0
162    0.0
194    0.0
Name: agri_gard_land_quantity, dtype: float64
93       5.0
176    150.0
Name: agri_gard_water_harv_farmlands, dtype: float64
93       0.0
176    100.0
Name: agri_gard_use_quantity, dtype: float64
17.7192982456
4.65437788018
3.02857142857
0.488479262673
perc_rainfed_farmlands
0.000000    14
0.166667     1
0.200000     1
0.333333     1
0.400000     2
0.500000     3
0.750000     1
0.800000     1
1.000000    26
inf          4
dtype: int64
agri_gard_farm_tech
smart           4
traditional    53
dtype: int64
agri_gard_water_harv_farmlands
0.0     11
1.0      7
2.0      4
4.0      3
5.0      3
6.0      1
8.0      1
10.0     5
dtype: int64
agri_gard_use_quantity
0.0      2
1.0      3
2.0     10
3.0      2
4.0      3
5.0      5
6.0      2
8.0      1
10.0     7
dtype: int64
agri_gard_land_property
community      2
none           2
own           2

agri_gard_training
no    57
dtype: int64

#### Climate change awareness

In [ ]:
cc_knowledge = reg_survey.groupby('cc_knowledge').size()
taking_action_cc = reg_survey.groupby('taking_action_cc').size()
fig_cc_knowledge = {
    "layout": {
        "title": {
            "text": "Households declaring CC knowledge"
        },
    },
    "data": [
        {

            "values": cc_knowledge.values,
            "labels": cc_knowledge.index.values,
            "domain": {
                "x": [0, .48]
            },
            "type": "pie",
            "sort": False
        },
        {

            "values": taking_action_cc.values,
            "labels": taking_action_cc.index.values,
            "domain": {
                "x": [.52, 1]
            },
            "type": "pie",
            "sort": False
        },
    ]
}

iplot(fig_cc_knowledge)

#### EWS awareness

In [ ]:
ews_knowledge = reg_survey.groupby('ews_knowledge').size()
ews_use = reg_survey.groupby('use_ews').size()
fig_ews = {
    "layout": {
        "title": {
            "text": "Households declaring EWS use"
        },
    },
    "data": [
        {

            "values": ews_knowledge.values,
            "labels": ews_knowledge.index.values,
            "domain": {
                "x": [0, .48]
            },
            "type": "pie",
            "sort": False
        },
        {

            "values": ews_use.values,
            "labels": ews_use.index.values,
            "domain": {
                "x": [.52, 1]
            },
            "type": "pie",
            "sort": False
        },
    ]
}

iplot(fig_ews)

#### Renewable energy awareness

In [ ]:
renewable_knowledge = reg_survey.groupby('renewable_knowledge').size()
solar_energy = reg_survey.groupby('solar_energy').size()
fig_renewable_knowledge = {
    "layout": {
        "title": {
            "text": "Households declaring CC knowledge"
        },
    },
    "data": [
        {

            "values": renewable_knowledge.values,
            "labels": renewable_knowledge.index.values,
            "domain": {
                "x": [0, .48]
            },
            "type": "pie",
            "sort": False
        },
        {
            "values": solar_energy.values,
            "labels": solar_energy.index.values,
            "domain": {
                "x": [.52, 1]
            },
            "type": "pie",
            "sort": False
        },
    ]
}

iplot(fig_renewable_knowledge)

#### Cooking facilities

In [ ]:
kitchen_type = reg_survey.groupby('kitchen_type').size()
kitchen = pd.crosstab(reg_survey.village, reg_survey.kitchen_type)
display(kitchen)
education = reg_survey.groupby('education').size()
fig_kitchen_type = {
    "layout": {
        "title": {
            "text": "Households declaring CC knowledge"
        },
    },
    "data": [
        {

            "values": kitchen_type.values,
            "labels": kitchen_type.index.values,
            "domain": {
                "x": [0, .48]
            },
            "type": "pie",
            "sort": False
        },
        {
            "values": education.values,
            "labels": education.index.values,
            "domain": {
                "x": [.52, 1]
            },
            "type": "pie",
            "sort": False
        },
    ]
}

iplot(fig_kitchen_type)

## Climate change effects

In [ ]:
cc_effects_on_rain = get_ocurrences(reg_survey, 'changed_rainy_season')

print cc_effects_on_rain

In [ ]:
sand_mov_sums = reg_survey.groupby('sand_movements').size()
wind_changes = get_ocurrences(reg_survey, 'sand_movements')

print wind_changes
print sand_mov_sums

In [ ]:
temps_changes = get_ocurrences(reg_survey, 'temps_change')

print temps_changes